In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage
from langchain_core.tools import tool
from tavily import TavilyClient
from typing import Dict, Any
import base64

In [ ]:
@tool
def web_search(query: str) -> Dict[str, Any]:
    """Search the web for information"""
    return tavily_client.search(query)


MODEL_NAME = "gemini-2.5-flash-lite"

tavily_client = TavilyClient()

model = init_chat_model(model=MODEL_NAME, model_provider="google_genai")

config = {"configurable": {"thread_id": "1"}}

agent = create_agent(model=model,
                     checkpointer=InMemorySaver(),
                     tools=[web_search])

## Upload fridge image

In [ ]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.png', multiple=False)
display(uploader)

In [ ]:
uploaded_file = uploader.value[0]

content_mv = uploaded_file["content"]
img_bytes = bytes(content_mv)
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me about the image"},
    {"type": "image", "base64": img_b64, "mime_type": "image/png"}
])

## Record audio instruction

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import io
import time
from tqdm import tqdm

duration = 5  # seconds
sample_rate = 44100

print("Recording...")
audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
for _ in tqdm(range(duration * 10)):
    time.sleep(0.1)
sd.wait()
print("Done.")

buf = io.BytesIO()
write(buf, sample_rate, audio)
wav_bytes = buf.getvalue()

aud_b64 = base64.b64encode(wav_bytes).decode("utf-8")

audio_question = HumanMessage(content=[
    {"type": "text", "text": "Write the transcript of the user's voice"},
    {"type": "audio", "base64": aud_b64, "mime_type": "audio/wav"}
])

## Invoke agent

In [ ]:
userInstructionResponse = agent.invoke(
    {"messages": [multimodal_question][-1].content}
)

In [ ]:
image_question = HumanMessage(content=[
    {"type": "text", "text": [multimodal_question][-1].content},
    {"type": "image", "base64": img_b64, "mime_type": "image/png"}
])

userImageResponse = agent.invoke(
    {"messages": [image_question][-1].content}
)